<a href="https://colab.research.google.com/github/sanrajbandre/parallel-computing.github.io/blob/parallel-computing-colab/colab/parallel_computing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# DEVICE VERIFICATION

!nvidia-smi
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
#if device_name != b'Tesla K80':
if device_name != b'Tesla P100-PCIE-16GB':
  raise Exception("Unfortunately this instance does not have a Tesla K80...!")
else:
  print('GPU DEVICE GETS VERIFIED...!')



Fri Jul 10 21:35:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
print(device_name)

b'Tesla P100-PCIE-16GB'


In [ ]:
# IMPORT 
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('demo.csv')
df

In [13]:
! pip3 install pycuda

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 71kB 6.1MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4537050 sha256=421fd52ae9086759a72b569d8fa15709d5aafe524917bcbd979a89c147c38dc8
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.3.1-py2.py3-none-any.whl size=65243 sha256=7a562200a8b1e2df18bbbce2ffd6e21751530ab562282eb99ac74e6ebf94c910
  Stored in directory: /root/.cache/pip/wheels/ac/01/07/fe484d8aae398b7f6150dae468a53887f0e7e265f1fc7fc975
Successfully built pycuda pytools


In [19]:
import pycuda.gpuarray as gpuarray
import pycuda.driver as cuda
import pycuda.autoinit
import numpy
a_gpu = gpuarray.to_gpu(numpy.random.randn(1,4).astype(numpy.float64))
a_doubled = (2*a_gpu).get()

print (a_gpu)
print (a_doubled)

[[ 0.84765245 -0.22321006  1.14053563  0.07649727]]
[[ 1.69530491 -0.44642013  2.28107125  0.15299453]]


In [20]:
! conda install cupy

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.4
  latest version: 4.8.3

Please update conda by running

    $ conda update -n ba

In [21]:
    import pycuda.gpuarray as gpuarray
    import pycuda.driver as cuda
    from pycuda.compiler import SourceModule
    import pycuda.autoinit
    import pycuda.tools
    import pycuda.cumath
    from pycuda.elementwise import ElementwiseKernel
    import numpy
    import pycuda.driver as drv
    import math
    import sys


    ##Based off of http://wiki.tiker.net/PyCuda/Examples/SimpleSpeedTest

    ##CUDA CODE##
    #  This takes a float array, and calls 'cos' on every element in it
    #  NOTE THE C CODE
    #
    mod = SourceModule("""
        __global__ void cos_gpu_v2(float *out, float *a)
        {
/***************
*This is the big change in the CUDA code, we made it so that our index counts for what block number we are in.
*Also we have to use c/c++ style comments here, not python
*************/
            const int idx = blockDim.x*blockIdx.x+threadIdx.x; 
               a[idx] = cos(a[idx]);
            out[idx] = a[idx];
          }
      """)



    #size = input("Enter a value for block size:  ")    #for manual input
    block_range = [64,128,256,512,1024,2048,4096]    #to test block sizes of power of 2, no need for this to be power of 2
    for blocks in block_range:
        block_size = 128            #can be 1-max thread number (1028 or 512 for V2+ or V1 )
        nbr_tot = blocks * block_size        #total items we will calculate

        print ("Total number of elements: ", nbr_tot)
       
        ## for timing ##
        secgpu = []
        seccpu = []


        #Time to try it on the GPU#
        for i in range(0,100):        ##this is to get a few runs in to avrage out times
            a = numpy.random.randn(nbr_tot).astype(numpy.float32)      #make a random number, as a float, in a 4*4 array
            a_cos = numpy.empty_like(a)       
            ##Lets time this!##
            start = drv.Event()
            end = drv.Event()
            start.record()
            ##timing set up!#

   
            func = mod.get_function("cos_gpu_v2")            #retrieves the GPU function doublify, and stores it as func in python
            func(cuda.Out(a_cos),cuda.In(a), grid=(blocks,1),block=(block_size,1,1))        #calls the function, and this time we are not doing distinct separate calls to put data onto the gpu or pull it off, but rather using the nicer PyCUDA wrapper around it


            ##end time!##
            end.record()
            end.synchronize()
            secs = start.time_till(end)
            secgpu.append(secs)
            ##timer end##

        #Time to try it on the CPU#
        for i in range(0,100):        ##this is to get a few runs in to avrage out times
            b = numpy.random.randn(nbr_tot).astype(numpy.float32)      #make a random number, as a float, in a 4*4 array
            b_cos = numpy.empty_like(a)
            ##Lets time this!##
            start = drv.Event()
            end = drv.Event()
            start.record()
            ##timing set up!#
            #for i in range(0,nbr_tot):        ##for if you want to do the slow version still, but at these numbers even the numpy version is faster
            #    b_cos[i] =  math.cos(b[i])
            b_cos = numpy.cos(b)
       

            ##end time!##
            end.record()
            end.synchronize()
            secs = start.time_till(end)
            seccpu.append(secs)
            ##timer end##

        print ("The first four elements in the")
        print ("GPU")           
        for i in range(0,5):   
            print ("Value is: ",a[i],"  And the cos is: ", a_cos[i])        ##print the first 5 to show that it is truly doing what I say it is
        print ("CPU")
        for i in range(0,5):   
            print ("Value is: ",b[i],"  And the cos is: ", b_cos[i])        ##print the first 5 to show that it is truly doing what I say it is
        print ("Avg time for on GPU")
        avggpu = sum(secgpu) / len(secgpu)
        print (avggpu)
        print ("Avg time for on CPU")
        avgcpu = sum(seccpu) / len(seccpu)
        print (avgcpu)


Total number of elements:  8192
The first four elements in the
GPU
Value is:  1.2192714   And the cos is:  0.34432986
Value is:  1.3005776   And the cos is:  0.2669422
Value is:  -0.6173682   And the cos is:  0.8154048
Value is:  -0.76348114   And the cos is:  0.7224334
Value is:  -2.1744006   And the cos is:  -0.56761354
CPU
Value is:  -0.40739024   And the cos is:  0.918158
Value is:  1.2148414   And the cos is:  0.34848562
Value is:  1.2553872   And the cos is:  0.31020546
Value is:  -0.2135905   And the cos is:  0.97727615
Value is:  -0.1289837   And the cos is:  0.99169314
Avg time for on GPU
0.1388112001121044
Avg time for on CPU
0.08651616007089614
Total number of elements:  16384
The first four elements in the
GPU
Value is:  -2.1738317   And the cos is:  -0.56714505
Value is:  -0.9829846   And the cos is:  0.55454135
Value is:  -0.99775976   And the cos is:  0.542186
Value is:  0.09174022   And the cos is:  0.99579483
Value is:  -0.5820569   And the cos is:  0.83533365
CPU
Valu